In [3]:
import numpy
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,Flatten,Input,Lambda
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical,plot_model

from tqdm.notebook import tqdm

# dir
Base_dir = 'reso'

In [5]:
# image features extraction model
imageSize = [224,224]
model = VGG16()
model = Model(inputs=model.inputs,outputs=model.layers[-2].output) # leavinf out the pred layer
print(model.summary())

2022-06-20 13:40:39.265059: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
# all the images features are predected and saved in a .pkl file for safty
features ={}
directory = os.path.join(Base_dir,'Images')
# # --------------------------------------------------------------------------------------------------------this is too much time takking
# # print(os.listdir(directory))
# # for img_name in tqdm(os.listdir(directory)):
# for img_name in os.listdir(directory):
#     img_path = directory+'/'+img_name  # img path
#     img = load_img(img_path,target_size=imageSize) #img
#     img = np.array(img) # img --> numpy array
#     img = img.reshape(1,img.shape[0],img.shape[1],img.shape[2]) # resizzing for the model
#     img = preprocess_input(img) # vgg preproc
#     feature = model.predict(img,verbose=2) # getting features
#     img_id = img_name.split('.')[0] # getting the id of the image
#     features[img_id] = feature # store features

# with open('features.pkl', 'wb') as file: # save the features from above
#     pickle.dump(features, file)
# # --------------------------------------------------------------------------------------------------------

# to load features
with open('features.pkl','rb') as f:
    features = pickle.load(f)

len(features)

8091

In [7]:
# all captions for the images are loaded here
with open(os.path.join(Base_dir,'captions.txt'),'r') as f:
    next(f)
    captions_doc = f.read()

len(captions_doc)

In [60]:
len(captions_doc)

3319280

In [66]:
len(captions_doc.split('\n'))

40456

In [46]:
# mapping of image to captions
mapping = {}
for line in tqdm(captions_doc.split('\n')):
    tokens = line.split(',') # img, id --> line format
    if len(line)<2: # for errors
        continue
    img_id, caption = tokens[0],tokens[1:]
    img_id = img_id.split('.')[0]
    caption = " ".join(caption)
    if img_id not in mapping:
        mapping[img_id]=[]
    mapping[img_id].append(caption)

# saving one for safty
# with open('mapping.pkl', 'wb') as file: # save the mapping from above
#     pickle.dump(mapping, file)
# mapping
     
print(len(mapping))

#cleaninf the text data
def clean(mapping):
    for im,cap_s in mapping.items():
        for i in range(len(cap_s)):
            cap = cap_s[i]
            cap=cap.lower()
            cap=cap.replace('[^A-Za-z]','') #removings waste units
            cap=cap.replace('\s+',' ') # removinfg sapce
            cap = '<start> ' + " ".join([word for word in cap.split() if len(word)>1]) + ' <end>' # for the start and end tag && removing the worlds like "a" and others
            cap_s[i] = cap

print('before clean :\n',mapping['1000268201_693b08cb0e']) # mapping.keys() # 1000268201_693b08cb0e
clean(mapping)
print('after clean :\n',mapping['1000268201_693b08cb0e'])

  0%|          | 0/40456 [00:00<?, ?it/s]

8091
before clean :
 ['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .']
after clean :
 ['<start> child in pink dress is climbing up set of stairs in an entry way <end>', '<start> girl going into wooden building <end>', '<start> little girl climbing into wooden playhouse <end>', '<start> little girl climbing the stairs to her playhouse <end>', '<start> little girl in pink dress going into wooden cabin <end>']


In [58]:
# LSTM

# tokenizer
all_caption=[]
for key in mapping:
    for caption in mapping[key]:
        all_caption.append(caption)

# tokenizing teh text

toki = Tokenizer()
toki.fit_on_texts(all_caption)

# para
vocab_size=len(toki.word_index)+1 # for the iput to the nn
max_length= 5  # --> max legth fir padding
for i in all_caption:
    max_length=max(len(i.split()),max_length)
# max_length

In [1]:
# # train test split
# from sklearn.model_selection import train_test_split
# img_train,cap_train,img_test,cap_test=train_test_split(img_id,captions_doc,text_size=0.2,random_state=101)

In [76]:
list(mapping.keys())

['1000268201_693b08cb0e',
 '1001773457_577c3a7d70',
 '1002674143_1b742ab4b8',
 '1003163366_44323f5815',
 '1007129816_e794419615',
 '1007320043_627395c3d8',
 '1009434119_febe49276a',
 '1012212859_01547e3f17',
 '1015118661_980735411b',
 '1015584366_dfcec3c85a',
 '101654506_8eb26cfb60',
 '101669240_b2d3e7f17b',
 '1016887272_03199f49c4',
 '1019077836_6fc9b15408',
 '1019604187_d087bf9a5f',
 '1020651753_06077ec457',
 '1022454332_6af2c1449a',
 '1022454428_b6b660a67b',
 '1022975728_75515238d8',
 '102351840_323e3de834',
 '1024138940_f1fefbdce1',
 '102455176_5f8ead62d5',
 '1026685415_0431cbf574',
 '1028205764_7e8df9a2ea',
 '1030985833_b0902ea560',
 '103106960_e8a41d64f8',
 '103195344_5d2dc613a3',
 '103205630_682ca7285b',
 '1032122270_ea6f0beedb',
 '1032460886_4a598ed535',
 '1034276567_49bb87c51c',
 '104136873_5b5d41be75',
 '1042020065_fb3d3ba5ba',
 '1042590306_95dea0916c',
 '1045521051_108ebc19be',
 '1048710776_bb5b0a5c7c',
 '1052358063_eae6744153',
 '105342180_4d4a40b47f',
 '1053804096_ad278b25

In [75]:
captions_doc.split('\n')

['1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .',
 '1000268201_693b08cb0e.jpg,A girl going into a wooden building .',
 '1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .',
 '1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playhouse .',
 '1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a wooden cabin .',
 '1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting',
 '1001773457_577c3a7d70.jpg,A black dog and a tri-colored dog playing with each other on the road .',
 '1001773457_577c3a7d70.jpg,A black dog and a white dog with brown spots are staring at each other in the street .',
 '1001773457_577c3a7d70.jpg,Two dogs of different breeds looking at each other on the road .',
 '1001773457_577c3a7d70.jpg,Two dogs on pavement moving toward each other .',
 '1002674143_1b742ab4b8.jpg,A little girl covered in paint sits in front of a painted rainbow with her hands in a 